In [90]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import colors, cm, pyplot as plt
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

In [91]:
CONVERTED_DATABASE = '/Users/darylwilding-mcbride/Downloads/190719_Hela_Ecoli/converted/190719_Hela_Ecoli_1to1_01-converted.sqlite'
MS1_CE = 10

In [92]:
db_conn = sqlite3.connect(CONVERTED_DATABASE)
ms1_frame_properties_df = pd.read_sql_query("select frame_id,retention_time_secs from frame_properties where collision_energy == {}".format(MS1_CE), db_conn)
ms1_frame_ids = tuple(ms1_frame_properties_df.frame_id)
db_conn.close()

In [123]:
PIXELS_X = 910
PIXELS_Y = 910  # equal to the number of scan lines
PIXELS_PER_BIN = 1

MZ_MIN = 100.0
MZ_MAX = 1700.0
SCAN_MAX = PIXELS_Y
SCAN_MIN = 1

In [124]:
MZ_PER_TILE = 18.0
MZ_BIN_WIDTH = MZ_PER_TILE / (PIXELS_X * PIXELS_PER_BIN)

In [125]:
MZ_BIN_WIDTH

0.01978021978021978

In [126]:
TILES_PER_FRAME = int((MZ_MAX - MZ_MIN) / MZ_PER_TILE)
TILES_PER_FRAME

88

In [127]:
mz_bins = np.arange(start=MZ_MIN, stop=MZ_MAX+MZ_BIN_WIDTH, step=MZ_BIN_WIDTH)  # go slightly wider to accommodate the maximum value
MZ_BIN_COUNT = len(mz_bins)
MZ_BIN_COUNT

80890

In [128]:
# choose a frame to test
FRAME_ID = ms1_frame_ids[int(len(ms1_frame_ids) / 2)]
FRAME_ID

5944

In [129]:
db_conn = sqlite3.connect(CONVERTED_DATABASE)
raw_points_df = pd.read_sql_query("select mz,scan,intensity from frames where frame_id == {}".format(FRAME_ID), db_conn)
db_conn.close()

In [130]:
frame_intensity_array = np.zeros([SCAN_MAX+1, MZ_BIN_COUNT+1], dtype=np.uint16)  # scratchpad for the intensity value prior to image conversion
for r in zip(raw_points_df.mz,raw_points_df.scan,raw_points_df.intensity):
    mz = r[0]
    scan = int(r[1])
    if (mz >= MZ_MIN) and (mz <= MZ_MAX) and (scan >= SCAN_MIN) and (scan <= SCAN_MAX):
        mz_array_idx = int(np.digitize(mz, mz_bins))-1
        scan_array_idx = scan
        intensity = int(r[2])
        frame_intensity_array[scan_array_idx,mz_array_idx] += intensity


In [131]:
# calculate the colour to represent the intensity
colour_map = cm.get_cmap(name='magma')
norm = colors.LogNorm(vmin=1, vmax=5e3, clip=True)  # aiming to get good colour variation in the lower range, and clipping everything else


In [132]:
# convert the intensity array to a dataframe
intensity_df = pd.DataFrame(frame_intensity_array).stack().rename_axis(['y', 'x']).reset_index(name='intensity')
# remove all the zero-intensity elements
intensity_df = intensity_df[intensity_df.intensity > 0]

# calculate the colour to represent the intensity
colour_l = []
for r in zip(intensity_df.intensity):
    colour_l.append((colour_map(norm(r[0]), bytes=True)[:3]))
intensity_df['colour'] = colour_l

In [133]:
# create an image of the whole frame
frame_im_array = np.zeros([PIXELS_Y+1, MZ_BIN_COUNT+1, 3], dtype=np.uint8)  # container for the image
for r in zip(intensity_df.x, intensity_df.y, intensity_df.colour):
    x = r[0]
    y = r[1]
    c = r[2]
    frame_im_array[y,x,:] = c

In [139]:
# extract the pixels for the frame
tile_idx = 33
tile_idx_base = tile_idx * PIXELS_X
tile_idx_width = PIXELS_X
# extract the subset of the frame for this image
tile_im_array = frame_im_array[:,tile_idx_base:tile_idx_base+tile_idx_width,:]
tile = Image.fromarray(tile_im_array, 'RGB')
tile.show()

In [137]:
mz_lower = MZ_MIN + (tile_idx * MZ_PER_TILE)
mz_upper = mz_lower + MZ_PER_TILE

In [138]:
mz_lower, mz_upper

(694.0, 712.0)

In [141]:
tile.save('/Users/darylwilding-mcbride/Downloads/frame-{}-tile-{}-mz-{}-{}.png'.format(FRAME_ID, tile_idx, int(mz_lower), int(mz_upper)))